In [ ]:
#query for efi2 deficit for Hearing impairment, using updated code for efi2 deficits
 CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efalls_hearing_impairment` as (
#bring all snoemd and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    cast(j.first_episodestartdate as datetime) first_episodestartdate
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for Hearing impairment snomed and ctv3code codes
  b as (
    select
    person_id,
    dateevent,
    case 
    when snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Hearing impairment' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate AND datetime_diff(first_episodestartdate,dateevent,year) <= 5 then 'true' 
    when ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Hearing impairment' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate AND datetime_diff(first_episodestartdate,dateevent,year) <= 5 then 'true' 
    when snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Hearing impairment' and TimeConstraintyears is null) AND dateevent < first_episodestartdate then 'true' 
    when ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Hearing impairment' and TimeConstraintyears is null) AND dateevent < first_episodestartdate then 'true' end as hearing_impairment
    from a_master)
    
    select
    distinct person_id,
    hearing_impairment,
    from b
    where hearing_impairment is not null)
  



